# RELANCEZ BIEN LE NOTEBOOK ENTIER A CHAQUE TRAINING POUR GARDER UNE TRACE DES RESULTATS DE CHAQUE MODEL

le model.summary est saved dans saved_models/nom_du_run_wandb

In [1]:
import sys
sys.path.append('../data_cleaning')

from time_series_forecasting import create_dataset
from dataset import build_dataset

In [23]:
import wandb
import tensorflow as tf
import tensorflow_addons as tfa

ModuleNotFoundError: No module named 'tensorflow_addons'

In [25]:
from tensorflow.keras.layers import Dense
from keras.layers import Input,Conv1D,MaxPooling1D,Bidirectional,Dropout,CuDNNLSTM,Dense,Input,LSTM, LeakyReLU
from tensorflow.keras import Sequential

### create wandb_api_key.txt and paste your api key inside &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; from https://wandb.ai/settings

In [7]:
file = open('wandb_api_key.txt', 'r')
wandb.login(key=file.readline()) # mettez
file.close()
            

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\theo-/.netrc


In [33]:
wandb.init(project="AI_Based_Sensor_Information_Fusion", entity="efrai")
batch_size = 128
epochs = 10
learning_rate = 1e-5
##To edit: 
wandb.config = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size,
    "window_size": 30,
    "stride":5,
    "number_imu":3,
    "bidirectional":True,
    "lstm_number": 2,
    "lstm_neurons":128,
    "dropout":0.2,
    "dense_number":1,
    "dense_first_neuron":64,#puissance de 2
    "activation_dense":'relu',
}

epoch,▁█
loss,██▇▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁
best_epoch,1
best_val_loss,0.04743
epoch,1
loss,0.05845
val_loss,0.04743


In [9]:
x_train, y_train = build_dataset(0, 50, wandb.config['window_size'],wandb.config['stride'],wandb.config['number_imu'])
x_test, y_test = build_dataset(51,60, wandb.config['window_size'],wandb.config['stride'],wandb.config['number_imu']) 

In [35]:
model = Sequential()
model.add(Input((wandb.config['window_size'], 6*wandb.config['number_imu'])))
if wandb.config['bidirectional']:
    model.add(Bidirectional(CuDNNLSTM(wandb.config['lstm_neurons'], return_sequences=True)))
else:
    model.add(LSTM(wandb.config['lstm_neurons'], return_sequences=True))
for s in range(wandb.config['lstm_number']-1):
    if wandb.config['bidirectional']:
        model.add(Bidirectional(CuDNNLSTM(wandb.config['lstm_neurons'])))
    else:
        model.add(CuDNNLSTM(wandb.config['lstm_neurons']))
model.add(Dropout(wandb.config['dropout']))
for d in range(wandb.config['dense_number']):
    model.add(Dense(int(wandb.config['dense_first_neuron'])/(d+1)**2, activation=wandb.config['activation_dense']))
model.add(Dense(6, activation='relu'))
    
    

opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=opt, loss='mse')  

In [36]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_10 (Bidirecti  (None, 30, 256)          151552    
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 256)              395264    
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                16448     
                                                                 
 dense_7 (Dense)             (None, 6)                 390       
                                                                 
Total params: 563,654
Trainable params: 563,654
Non-tr

In [37]:
model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size = wandb.config['batch_size'],
    epochs=wandb.config['epochs'],
    callbacks=[wandb.keras.WandbCallback(log_batch_frequency=5)])

Epoch 1/2
 50/350 [===>..........................] - ETA: 14s - loss: 0.3478 - _timestamp: 1648138385.0000 - _runtime: 49.0000

KeyboardInterrupt: 

In [5]:
wandb.finish()

In [16]:
y_pred = model.predict(x_test)

In [18]:
from sklearn.metrics import r2_score

for i in range(6):
    print(r2_score(y_test[:][i], y_pred[:][i]))

0.6716539918035276
0.5511174247136768
0.8319051887521381
0.3064378605436666
0.7801679626204026
0.8349222258691333
